In [17]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

In [8]:
df_train = pd.read_csv('training.csv')

In [9]:
df_test = pd.read_csv('testing.csv')

In [10]:
df_train = df_train.drop(columns=['Unnamed: 0', 'median_days_on_market', 'rent50_0', 'Unemployment', 'Employment', 
                                           'Unemployment rate', 'pct_foreign_born', 'active_listing_count_yy', 'TOT_BENES'])
df_train

,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,Labor force,median_hh_inc,pct_bb,...,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES,rent50_1,rent50_2,rent50_3,rent50_4,value
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,26496.0,55317.0,76.6,...,56.0,8241.0,2403.0,9892.0,7088.0,743.0,880.0,1112.0,1532.0,0.0514
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,26354.0,55317.0,76.6,...,56.0,8241.0,2403.0,9892.0,7088.0,743.0,880.0,1112.0,1532.0,0.0514
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,26526.0,55317.0,76.6,...,54.0,8241.0,2403.0,9892.0,7088.0,743.0,880.0,1112.0,1532.0,0.0514
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,26485.0,55317.0,76.6,...,32.0,8241.0,2403.0,9892.0,7088.0,743.0,880.0,1112.0,1532.0,0.0514
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,26543.0,55317.0,76.6,...,24.0,8241.0,2403.0,9892.0,7088.0,743.0,880.0,1112.0,1532.0,0.0514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,NaN,53333.0,79.7,...,0.0,1555.0,158.0,1588.0,1133.0,757.0,996.0,1231.0,1705.0,0.0000
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,NaN,53333.0,79.7,...,12.0,1555.0,158.0,1588.0,1133.0,757.0,996.0,1231.0,1705.0,0.0000
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,NaN,53333.0,79.7,...,8.0,1555.0,158.0,1588.0,1133.0,757.0,996.0,1231.0,1705.0,0.0000
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,NaN,53333.0,79.7,...,10.0,1555.0,161.0,1591.0,1135.0,757.0,996.0,1231.0,1705.0,0.0000


In [13]:
df_test = df_test.drop(columns=['Unnamed: 0'])
df_test

,row_id,cfips,first_day_of_month_orig,year,county,state,Employment,Labor force,median_hh_inc,pct_bb,...,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES,rent50_1,rent50_2,rent50_3,rent50_4,value
0,1001_2022-11-01,1001,2022-11-01,2022,Autauga County,Alabama,25898.0,26474.0,57982.0,82.7,...,8.0,9059.0,2390.0,10582.0,7861.0,816.0,976.0,1234.0,1596.0,0.0605
1,1003_2022-11-01,1003,2022-11-01,2022,Baldwin County,Alabama,97017.0,99271.0,61756.0,85.1,...,336.0,50778.0,7057.0,54120.0,43596.0,878.0,1149.0,1523.0,1938.0,0.0605
2,1005_2022-11-01,1005,2022-11-01,2022,Barbour County,Alabama,7741.0,8081.0,34990.0,64.6,...,10.0,4816.0,1396.0,5872.0,4958.0,562.0,739.0,914.0,1027.0,0.0605
3,1007_2022-11-01,1007,2022-11-01,2022,Bibb County,Alabama,8416.0,8638.0,51721.0,76.1,...,0.0,3470.0,1212.0,4387.0,3604.0,895.0,1029.0,1331.0,1436.0,0.0605
4,1009_2022-11-01,1009,2022-11-01,2022,Blount County,Alabama,25016.0,25500.0,48922.0,79.6,...,22.0,10680.0,2549.0,12432.0,10648.0,895.0,1029.0,1331.0,1436.0,0.0605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,2023,Sweetwater County,Wyoming,NaN,NaN,76668.0,88.4,...,14.0,6060.0,930.0,6473.0,4894.0,853.0,963.0,1315.0,1640.0,0.0000
25076,56039_2023-06-01,56039,2023-06-01,2023,Teton County,Wyoming,NaN,NaN,94498.0,90.5,...,16.0,3906.0,112.0,3701.0,2906.0,1303.0,1644.0,2316.0,2800.0,0.0000
25077,56041_2023-06-01,56041,2023-06-01,2023,Uinta County,Wyoming,NaN,NaN,75106.0,90.6,...,16.0,3458.0,532.0,3657.0,2865.0,666.0,877.0,1208.0,1465.0,0.0000
25078,56043_2023-06-01,56043,2023-06-01,2023,Washakie County,Wyoming,NaN,NaN,62271.0,85.4,...,12.0,1886.0,168.0,1879.0,1345.0,668.0,877.0,1246.0,1331.0,0.0000


In [14]:
df_train.columns

Index(['row_id', 'cfips', 'county', 'state', 'first_day_of_month_orig',
       'microbusiness_density', 'active', 'Labor force', 'median_hh_inc',
       'pct_bb', 'pct_college', 'pct_it_workers', 'active_listing_count',
       'new_listing_count', 'price_reduced_count', 'AGED_TOT_BENES',
       'DSBLD_TOT_BENES', 'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES',
       'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4', 'value'],
      dtype='object')

In [21]:
x_values = df_train[['Labor force', 'median_hh_inc',
       'pct_bb', 'pct_college', 'pct_it_workers', 'active_listing_count',
       'new_listing_count', 'price_reduced_count', 'AGED_TOT_BENES',
       'DSBLD_TOT_BENES', 'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES',
       'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4', 'value']]
x_values_cols = ['Labor force', 'median_hh_inc',
       'pct_bb', 'pct_college', 'pct_it_workers', 'active_listing_count',
       'new_listing_count', 'price_reduced_count', 'AGED_TOT_BENES',
       'DSBLD_TOT_BENES', 'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES',
       'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4', 'value']
y_values = df_train[['microbusiness_density']]

In [22]:
imp = SimpleImputer(strategy='mean', fill_value=0)
x_values = imp.fit_transform(x_values)
x_values = pd.DataFrame(x_values, columns=x_values_cols)
x_values

,Labor force,median_hh_inc,pct_bb,pct_college,pct_it_workers,active_listing_count,new_listing_count,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES,rent50_1,rent50_2,rent50_3,rent50_4,value
0,26496.000000,55317.0,76.6,14.5,1.3,226.0,66.0,56.0,8241.0,2403.0,9892.0,7088.0,743.0,880.0,1112.0,1532.0,0.0514
1,26354.000000,55317.0,76.6,14.5,1.3,221.0,88.0,56.0,8241.0,2403.0,9892.0,7088.0,743.0,880.0,1112.0,1532.0,0.0514
2,26526.000000,55317.0,76.6,14.5,1.3,202.0,60.0,54.0,8241.0,2403.0,9892.0,7088.0,743.0,880.0,1112.0,1532.0,0.0514
3,26485.000000,55317.0,76.6,14.5,1.3,201.0,62.0,32.0,8241.0,2403.0,9892.0,7088.0,743.0,880.0,1112.0,1532.0,0.0514
4,26543.000000,55317.0,76.6,14.5,1.3,200.0,80.0,24.0,8241.0,2403.0,9892.0,7088.0,743.0,880.0,1112.0,1532.0,0.0514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,86054.410758,53333.0,79.7,12.7,0.0,45.0,12.0,0.0,1555.0,158.0,1588.0,1133.0,757.0,996.0,1231.0,1705.0,0.0000
122261,86054.410758,53333.0,79.7,12.7,0.0,50.0,24.0,12.0,1555.0,158.0,1588.0,1133.0,757.0,996.0,1231.0,1705.0,0.0000
122262,86054.410758,53333.0,79.7,12.7,0.0,62.0,12.0,8.0,1555.0,158.0,1588.0,1133.0,757.0,996.0,1231.0,1705.0,0.0000
122263,86054.410758,53333.0,79.7,12.7,0.0,55.0,10.0,10.0,1555.0,161.0,1591.0,1135.0,757.0,996.0,1231.0,1705.0,0.0000


In [24]:
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x_values, df_train['microbusiness_density'])

RandomForestRegressor(max_depth=2, random_state=0)

In [26]:
df_test_predictors = df_test[['Labor force', 'median_hh_inc',
       'pct_bb', 'pct_college', 'pct_it_workers', 'active_listing_count',
       'new_listing_count', 'price_reduced_count', 'AGED_TOT_BENES',
       'DSBLD_TOT_BENES', 'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES',
       'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4', 'value']]

In [28]:
imp = SimpleImputer(strategy='mean', fill_value=0)
x = imp.fit_transform(df_test_predictors)
x = pd.DataFrame(x, columns=x_values_cols)
x

,Labor force,median_hh_inc,pct_bb,pct_college,pct_it_workers,active_listing_count,new_listing_count,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES,rent50_1,rent50_2,rent50_3,rent50_4,value
0,26474.000000,57982.0,82.7,16.7,0.6,55.0,76.0,8.0,9059.0,2390.0,10582.0,7861.0,816.0,976.0,1234.0,1596.0,0.0605
1,99271.000000,61756.0,85.1,20.2,1.0,1233.0,794.0,336.0,50778.0,7057.0,54120.0,43596.0,878.0,1149.0,1523.0,1938.0,0.0605
2,8081.000000,34990.0,64.6,7.3,1.1,49.0,18.0,10.0,4816.0,1396.0,5872.0,4958.0,562.0,739.0,914.0,1027.0,0.0605
3,8638.000000,51721.0,76.1,7.4,1.7,9.0,6.0,0.0,3470.0,1212.0,4387.0,3604.0,895.0,1029.0,1331.0,1436.0,0.0605
4,25500.000000,48922.0,79.6,8.9,1.1,76.0,52.0,22.0,10680.0,2549.0,12432.0,10648.0,895.0,1029.0,1331.0,1436.0,0.0605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25075,87403.720684,76668.0,88.4,12.4,1.0,69.0,54.0,14.0,6060.0,930.0,6473.0,4894.0,853.0,963.0,1315.0,1640.0,0.0000
25076,87403.720684,94498.0,90.5,38.3,2.0,103.0,58.0,16.0,3906.0,112.0,3701.0,2906.0,1303.0,1644.0,2316.0,2800.0,0.0000
25077,87403.720684,75106.0,90.6,12.3,0.9,49.0,22.0,16.0,3458.0,532.0,3657.0,2865.0,666.0,877.0,1208.0,1465.0,0.0000
25078,87403.720684,62271.0,85.4,17.2,1.1,26.0,10.0,12.0,1886.0,168.0,1879.0,1345.0,668.0,877.0,1246.0,1331.0,0.0000


In [29]:
regr.predict(x)

array([4.11966722, 7.84236719, 2.10284393, ..., 2.18468871, 4.00900708,
       3.93813742])

In [31]:
df_submit = df_test[['row_id']]
df_submit['microbusiness_density']= regr.predict(x)

C:\Users\jacks\AppData\Local\Temp\ipykernel_34660\2925770040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submit['microbusiness_density']= regr.predict(x)


In [32]:
df_submit

,row_id,microbusiness_density
0,1001_2022-11-01,4.119667
1,1003_2022-11-01,7.842367
2,1005_2022-11-01,2.102844
3,1007_2022-11-01,2.146378
4,1009_2022-11-01,2.146378
...,...,...
25075,56037_2023-06-01,2.272964
25076,56039_2023-06-01,15.428494
25077,56041_2023-06-01,2.184689
25078,56043_2023-06-01,4.009007


In [35]:
df_submit.to_csv('submission_1.csv', index=False)

In [42]:
data = df_train[['Labor force', 'median_hh_inc',
       'pct_bb', 'pct_college', 'pct_it_workers', 'active_listing_count',
       'new_listing_count', 'price_reduced_count', 'AGED_TOT_BENES',
       'DSBLD_TOT_BENES', 'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES',
       'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4', 'value']]
x_values_2 = pd.concat([data, pd.get_dummies(df_train['state'])], axis=1)

x_values_2

,Labor force,median_hh_inc,pct_bb,pct_college,pct_it_workers,active_listing_count,new_listing_count,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,26496.0,55317.0,76.6,14.5,1.3,226.0,66.0,56.0,8241.0,2403.0,...,0,0,0,0,0,0,0,0,0,0
1,26354.0,55317.0,76.6,14.5,1.3,221.0,88.0,56.0,8241.0,2403.0,...,0,0,0,0,0,0,0,0,0,0
2,26526.0,55317.0,76.6,14.5,1.3,202.0,60.0,54.0,8241.0,2403.0,...,0,0,0,0,0,0,0,0,0,0
3,26485.0,55317.0,76.6,14.5,1.3,201.0,62.0,32.0,8241.0,2403.0,...,0,0,0,0,0,0,0,0,0,0
4,26543.0,55317.0,76.6,14.5,1.3,200.0,80.0,24.0,8241.0,2403.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,NaN,53333.0,79.7,12.7,0.0,45.0,12.0,0.0,1555.0,158.0,...,0,0,0,0,0,0,0,0,0,1
122261,NaN,53333.0,79.7,12.7,0.0,50.0,24.0,12.0,1555.0,158.0,...,0,0,0,0,0,0,0,0,0,1
122262,NaN,53333.0,79.7,12.7,0.0,62.0,12.0,8.0,1555.0,158.0,...,0,0,0,0,0,0,0,0,0,1
122263,NaN,53333.0,79.7,12.7,0.0,55.0,10.0,10.0,1555.0,161.0,...,0,0,0,0,0,0,0,0,0,1


In [44]:
imp = SimpleImputer(strategy='mean', fill_value=0)
x = imp.fit_transform(x_values_2)
x = pd.DataFrame(x, columns=x_values_2.columns)
x

,Labor force,median_hh_inc,pct_bb,pct_college,pct_it_workers,active_listing_count,new_listing_count,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,26496.000000,55317.0,76.6,14.5,1.3,226.0,66.0,56.0,8241.0,2403.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26354.000000,55317.0,76.6,14.5,1.3,221.0,88.0,56.0,8241.0,2403.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26526.000000,55317.0,76.6,14.5,1.3,202.0,60.0,54.0,8241.0,2403.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26485.000000,55317.0,76.6,14.5,1.3,201.0,62.0,32.0,8241.0,2403.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26543.000000,55317.0,76.6,14.5,1.3,200.0,80.0,24.0,8241.0,2403.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,86054.410758,53333.0,79.7,12.7,0.0,45.0,12.0,0.0,1555.0,158.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
122261,86054.410758,53333.0,79.7,12.7,0.0,50.0,24.0,12.0,1555.0,158.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
122262,86054.410758,53333.0,79.7,12.7,0.0,62.0,12.0,8.0,1555.0,158.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
122263,86054.410758,53333.0,79.7,12.7,0.0,55.0,10.0,10.0,1555.0,161.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [45]:
regr_2 = RandomForestRegressor(max_depth=4, random_state=0)
regr_2.fit(x, df_train['microbusiness_density'])

RandomForestRegressor(max_depth=4, random_state=0)

In [46]:
data = df_test[['Labor force', 'median_hh_inc',
       'pct_bb', 'pct_college', 'pct_it_workers', 'active_listing_count',
       'new_listing_count', 'price_reduced_count', 'AGED_TOT_BENES',
       'DSBLD_TOT_BENES', 'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES',
       'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4', 'value']]
x_values_test = pd.concat([data, pd.get_dummies(df_test['state'])], axis=1)

x_values_test

,Labor force,median_hh_inc,pct_bb,pct_college,pct_it_workers,active_listing_count,new_listing_count,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,26474.0,57982.0,82.7,16.7,0.6,55.0,76.0,8.0,9059.0,2390.0,...,0,0,0,0,0,0,0,0,0,0
1,99271.0,61756.0,85.1,20.2,1.0,1233.0,794.0,336.0,50778.0,7057.0,...,0,0,0,0,0,0,0,0,0,0
2,8081.0,34990.0,64.6,7.3,1.1,49.0,18.0,10.0,4816.0,1396.0,...,0,0,0,0,0,0,0,0,0,0
3,8638.0,51721.0,76.1,7.4,1.7,9.0,6.0,0.0,3470.0,1212.0,...,0,0,0,0,0,0,0,0,0,0
4,25500.0,48922.0,79.6,8.9,1.1,76.0,52.0,22.0,10680.0,2549.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25075,NaN,76668.0,88.4,12.4,1.0,69.0,54.0,14.0,6060.0,930.0,...,0,0,0,0,0,0,0,0,0,1
25076,NaN,94498.0,90.5,38.3,2.0,103.0,58.0,16.0,3906.0,112.0,...,0,0,0,0,0,0,0,0,0,1
25077,NaN,75106.0,90.6,12.3,0.9,49.0,22.0,16.0,3458.0,532.0,...,0,0,0,0,0,0,0,0,0,1
25078,NaN,62271.0,85.4,17.2,1.1,26.0,10.0,12.0,1886.0,168.0,...,0,0,0,0,0,0,0,0,0,1


In [49]:
imp = SimpleImputer(strategy='mean', fill_value=0)
x_test = imp.fit_transform(x_values_test)
x_test = pd.DataFrame(x_test, columns=x_values_test.columns)
x_test

,Labor force,median_hh_inc,pct_bb,pct_college,pct_it_workers,active_listing_count,new_listing_count,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,26474.000000,57982.0,82.7,16.7,0.6,55.0,76.0,8.0,9059.0,2390.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,99271.000000,61756.0,85.1,20.2,1.0,1233.0,794.0,336.0,50778.0,7057.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8081.000000,34990.0,64.6,7.3,1.1,49.0,18.0,10.0,4816.0,1396.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8638.000000,51721.0,76.1,7.4,1.7,9.0,6.0,0.0,3470.0,1212.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,25500.000000,48922.0,79.6,8.9,1.1,76.0,52.0,22.0,10680.0,2549.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25075,87403.720684,76668.0,88.4,12.4,1.0,69.0,54.0,14.0,6060.0,930.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25076,87403.720684,94498.0,90.5,38.3,2.0,103.0,58.0,16.0,3906.0,112.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25077,87403.720684,75106.0,90.6,12.3,0.9,49.0,22.0,16.0,3458.0,532.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25078,87403.720684,62271.0,85.4,17.2,1.1,26.0,10.0,12.0,1886.0,168.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [50]:
df_submit_2 = df_test[['row_id']]
df_submit_2['microbusiness_density']= regr_2.predict(x_test)

C:\Users\jacks\AppData\Local\Temp\ipykernel_34660\3480477833.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submit_2['microbusiness_density']= regr_2.predict(x_test)


In [53]:
df_submit_2.to_csv('submission_2.csv', index=False)

In [57]:
data = df_train[['Labor force', 'median_hh_inc',
       'pct_bb', 'pct_college', 'pct_it_workers', 'active_listing_count',
       'new_listing_count', 'price_reduced_count', 'AGED_TOT_BENES',
       'DSBLD_TOT_BENES', 'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES',
       'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4', 'value']]
x_values_train = pd.concat([data, pd.get_dummies(df_train['state']), pd.get_dummies(df_train['county'])], axis=1)

x_values_train

,Labor force,median_hh_inc,pct_bb,pct_college,pct_it_workers,active_listing_count,new_listing_count,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,...,Yoakum County,Yolo County,York County,Young County,Yuba County,Yukon-Koyukuk Census Area,Yuma County,Zapata County,Zavala County,Ziebach County
0,26496.0,55317.0,76.6,14.5,1.3,226.0,66.0,56.0,8241.0,2403.0,...,0,0,0,0,0,0,0,0,0,0
1,26354.0,55317.0,76.6,14.5,1.3,221.0,88.0,56.0,8241.0,2403.0,...,0,0,0,0,0,0,0,0,0,0
2,26526.0,55317.0,76.6,14.5,1.3,202.0,60.0,54.0,8241.0,2403.0,...,0,0,0,0,0,0,0,0,0,0
3,26485.0,55317.0,76.6,14.5,1.3,201.0,62.0,32.0,8241.0,2403.0,...,0,0,0,0,0,0,0,0,0,0
4,26543.0,55317.0,76.6,14.5,1.3,200.0,80.0,24.0,8241.0,2403.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,NaN,53333.0,79.7,12.7,0.0,45.0,12.0,0.0,1555.0,158.0,...,0,0,0,0,0,0,0,0,0,0
122261,NaN,53333.0,79.7,12.7,0.0,50.0,24.0,12.0,1555.0,158.0,...,0,0,0,0,0,0,0,0,0,0
122262,NaN,53333.0,79.7,12.7,0.0,62.0,12.0,8.0,1555.0,158.0,...,0,0,0,0,0,0,0,0,0,0
122263,NaN,53333.0,79.7,12.7,0.0,55.0,10.0,10.0,1555.0,161.0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
imp = SimpleImputer(strategy='mean', fill_value=0)
x = imp.fit_transform(x_values_train)
x = pd.DataFrame(x, columns=x_values_train.columns)
x

,Labor force,median_hh_inc,pct_bb,pct_college,pct_it_workers,active_listing_count,new_listing_count,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,...,Yoakum County,Yolo County,York County,Young County,Yuba County,Yukon-Koyukuk Census Area,Yuma County,Zapata County,Zavala County,Ziebach County
0,26496.000000,55317.0,76.6,14.5,1.3,226.0,66.0,56.0,8241.0,2403.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26354.000000,55317.0,76.6,14.5,1.3,221.0,88.0,56.0,8241.0,2403.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26526.000000,55317.0,76.6,14.5,1.3,202.0,60.0,54.0,8241.0,2403.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26485.000000,55317.0,76.6,14.5,1.3,201.0,62.0,32.0,8241.0,2403.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26543.000000,55317.0,76.6,14.5,1.3,200.0,80.0,24.0,8241.0,2403.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,86054.410758,53333.0,79.7,12.7,0.0,45.0,12.0,0.0,1555.0,158.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122261,86054.410758,53333.0,79.7,12.7,0.0,50.0,24.0,12.0,1555.0,158.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122262,86054.410758,53333.0,79.7,12.7,0.0,62.0,12.0,8.0,1555.0,158.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122263,86054.410758,53333.0,79.7,12.7,0.0,55.0,10.0,10.0,1555.0,161.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
regr_3 = RandomForestRegressor(max_depth=6, random_state=0)
regr_3.fit(x, df_train['microbusiness_density'])

RandomForestRegressor(max_depth=6, random_state=0)

In [66]:
data = df_test[['Labor force', 'median_hh_inc',
       'pct_bb', 'pct_college', 'pct_it_workers', 'active_listing_count',
       'new_listing_count', 'price_reduced_count', 'AGED_TOT_BENES',
       'DSBLD_TOT_BENES', 'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES',
       'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4', 'value']]
x_values_test_2 = pd.concat([data, pd.get_dummies(df_test['state']), pd.get_dummies(df_test['county'])], axis=1)

x_values_test_2

,Labor force,median_hh_inc,pct_bb,pct_college,pct_it_workers,active_listing_count,new_listing_count,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,...,Yoakum County,Yolo County,York County,Young County,Yuba County,Yukon-Koyukuk Census Area,Yuma County,Zapata County,Zavala County,Ziebach County
0,26474.0,57982.0,82.7,16.7,0.6,55.0,76.0,8.0,9059.0,2390.0,...,0,0,0,0,0,0,0,0,0,0
1,99271.0,61756.0,85.1,20.2,1.0,1233.0,794.0,336.0,50778.0,7057.0,...,0,0,0,0,0,0,0,0,0,0
2,8081.0,34990.0,64.6,7.3,1.1,49.0,18.0,10.0,4816.0,1396.0,...,0,0,0,0,0,0,0,0,0,0
3,8638.0,51721.0,76.1,7.4,1.7,9.0,6.0,0.0,3470.0,1212.0,...,0,0,0,0,0,0,0,0,0,0
4,25500.0,48922.0,79.6,8.9,1.1,76.0,52.0,22.0,10680.0,2549.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25075,NaN,76668.0,88.4,12.4,1.0,69.0,54.0,14.0,6060.0,930.0,...,0,0,0,0,0,0,0,0,0,0
25076,NaN,94498.0,90.5,38.3,2.0,103.0,58.0,16.0,3906.0,112.0,...,0,0,0,0,0,0,0,0,0,0
25077,NaN,75106.0,90.6,12.3,0.9,49.0,22.0,16.0,3458.0,532.0,...,0,0,0,0,0,0,0,0,0,0
25078,NaN,62271.0,85.4,17.2,1.1,26.0,10.0,12.0,1886.0,168.0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
imp = SimpleImputer(strategy='mean', fill_value=0)
x_test = imp.fit_transform(x_values_test_2)
x_test = pd.DataFrame(x_test, columns=x_values_test_2.columns)
x_test

,Labor force,median_hh_inc,pct_bb,pct_college,pct_it_workers,active_listing_count,new_listing_count,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,...,Yoakum County,Yolo County,York County,Young County,Yuba County,Yukon-Koyukuk Census Area,Yuma County,Zapata County,Zavala County,Ziebach County
0,26474.000000,57982.0,82.7,16.7,0.6,55.0,76.0,8.0,9059.0,2390.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,99271.000000,61756.0,85.1,20.2,1.0,1233.0,794.0,336.0,50778.0,7057.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8081.000000,34990.0,64.6,7.3,1.1,49.0,18.0,10.0,4816.0,1396.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8638.000000,51721.0,76.1,7.4,1.7,9.0,6.0,0.0,3470.0,1212.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,25500.000000,48922.0,79.6,8.9,1.1,76.0,52.0,22.0,10680.0,2549.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25075,87403.720684,76668.0,88.4,12.4,1.0,69.0,54.0,14.0,6060.0,930.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25076,87403.720684,94498.0,90.5,38.3,2.0,103.0,58.0,16.0,3906.0,112.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25077,87403.720684,75106.0,90.6,12.3,0.9,49.0,22.0,16.0,3458.0,532.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25078,87403.720684,62271.0,85.4,17.2,1.1,26.0,10.0,12.0,1886.0,168.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
df_submit_3 = df_test[['row_id']]
df_submit_3['microbusiness_density']= regr_3.predict(x_test)

C:\Users\jacks\AppData\Local\Temp\ipykernel_34660\3689491034.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submit_3['microbusiness_density']= regr_3.predict(x_test)


In [70]:
df_submit_3.to_csv('submission_3.csv', index = False)